In [ ]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

# 3. Tiền xử lí dữ liệu

In [2]:
save_dir=f"{exps_dir}/exp1/exp_gau"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [ ]:
test_size=0.33
seed=42
data=pd.read_excel(f'{exps_dir}/exp1/data_EDA.xlsx')
data

In [ ]:
data_train, data_test = train_test_split(data, test_size = 0.33, random_state=seed)
print(data_train.shape,data_test.shape)
data_train.to_excel(f'{save_dir}/datatrain.xlsx', index=None)
data_test.to_excel(f'{save_dir}/datatest.xlsx', index=None)

In [ ]:
data_train

In [6]:
# outliers_list=dict(np.load(f'{save_dir}/outliers.npz'))['outliers']
# def remove_outliers_iqr(data, column):
#     Q1 = data[column].quantile(0.25)
#     Q3 = data[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# # Áp dụng hàm cho các cột chứa dữ liệu cần loại bỏ ngoại lệ
# for column in outliers_list:
#     data_train = remove_outliers_iqr(data_train, column)
# data_train

In [ ]:
data.isnull().sum()

In [8]:
# data_train['collision_type']=data_train['collision_type'].replace('?',np.nan)
# data_train['collision_type'].fillna(method='ffill',inplace=True)
# data_train[['property_damage','police_report_available']]=data_train[['property_damage','police_report_available']].replace('?','Unknow')
# (data_train=='?').sum()

# 4.Chuẩn hóa dữ liệu

In [ ]:
numeric_columns=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['numeric_columns']
numeric_columns

In [ ]:
category_columns=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['category_columns']
category_columns

In [ ]:
label_encoders = {}
for column in category_columns:
    label_encoder = LabelEncoder()
    data_train[column] = label_encoder.fit_transform(data_train[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = StandardScaler()
data_train[numeric_columns] = scaler.fit_transform(data_train[numeric_columns])
data_train

In [12]:
x_train=data_train.drop('fraud_reported',axis=1)
y_train=data_train['fraud_reported']
x_train.to_excel(f'{save_dir}/x_train.xlsx', index=False)
y_train.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

In [14]:
# from imblearn.over_sampling import SMOTE
# x_train, y_train = SMOTE().fit_resample(x_train, y_train)
# x_train.to_excel(f'{save_dir}/x_train.xlsx', index=False)
# y_train.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(data_train.drop(columns=['fraud_reported']), data_train['fraud_reported'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [ ]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer



gau_synthesizer = GaussianCopulaSynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            default_distribution='gamma') 

gau_synthesizer.fit(oversampling_sample)
gau_synthesizer.save(
    filepath=f'{save_dir}/gau_synthesizer.pkl'
)

In [ ]:
gau_synthetic_data = gau_synthesizer.sample(num_rows=1000)
gau_synthetic_data.head()

In [ ]:
label_encoders = {}
for column in category_columns:
    label_encoder = LabelEncoder()
    gau_synthetic_data[column] = label_encoder.fit_transform(gau_synthetic_data[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = StandardScaler()
gau_synthetic_data[numeric_columns] = scaler.fit_transform(gau_synthetic_data[numeric_columns])


In [ ]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gau = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sdv.evaluation.single_table import evaluate_quality



diagnostic_gau = evaluate_quality(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## Gau data
gau_data=gau_synthetic_data[gau_synthetic_data.fraud_reported==1]
gau_sample=pd.concat([data_train, gau_data] , axis=0, ignore_index=True)
X_gausampled, y_gausampled = rus.fit_resample(gau_sample.drop(columns=['fraud_reported']), gau_sample['fraud_reported'])
X_gausampled = pd.DataFrame(X_gausampled)
y_gausampled = pd.DataFrame(y_gausampled)
gau_sample=pd.concat([X_gausampled, y_gausampled], axis=1)    
gau_sample = shuffle(gau_sample, random_state=42)
print(gau_sample.fraud_reported.value_counts())

In [ ]:
print(gau_sample)

In [ ]:
gau_sample.to_excel(f'{save_dir}/data_train.xlsx', index=False)

In [ ]:
feat_gau = gau_sample.drop(columns=['fraud_reported'])
target_gau = gau_sample['fraud_reported']

In [ ]:
feat_gau.to_excel(f'{save_dir}/x_train.xlsx', index=False)
target_gau.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
print(f"+ X_train: {len(x_train)}")
for fold, (train_idx, valid_idx) in enumerate(kfold.split(x_train, y_train)):
    print(f'Fold {fold}: ')
    print(f'+ train_idx: {train_idx}')
    print(f'+ valid_idx: {valid_idx}')
    print(f'+ train / valid: {valid_idx}')
    pass

In [16]:
object_cols_test=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['category_columns']
numeric_cols_test=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['numeric_columns']
# for column in outliers_list:
#     data_test = remove_outliers_iqr(data_test, column)
# data_test['collision_type'].replace('?',np.nan,inplace=True)
# data_test['collision_type'].fillna(method='ffill',inplace=True)
# data_test[['property_damage','police_report_available']].replace('?','Unknow',inplace=True)
# (data_test=='?').sum()
label_encoders = {}
for column in object_cols_test:
    label_encoder = LabelEncoder()
    data_test[column] = label_encoder.fit_transform(data_test[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = StandardScaler()
data_test[numeric_cols_test] = scaler.fit_transform(data_test[numeric_cols_test])

x_test=data_test.drop('fraud_reported',axis=1)
y_test=data_test['fraud_reported']
data_test.to_excel(f'{save_dir}/df_test_minmax.xlsx', index=False)
x_test.to_excel(f'{save_dir}/x_test.xlsx', index=False)
y_test.to_excel(f'{save_dir}/y_test.xlsx', index=False)